# Torus one-point functions of the $O(n)$ model

In [27]:
Pkg.activate("."); # activate the parent environment
using BootstrapVirasoro, BootstrapVirasoro.LoopModels, BenchmarkTools
println("Number of threads: $(Threads.nthreads())")

  Activating project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro/examples`


Number of threads: 4


In [2]:
const Sig = Channels{Rational}

# Spectrum of O(n) model
c = CC(β=sqrt(big"2")/(big"0.8"+big"0.2"*im))
diag_field = Field(c, r=0, s=big"0.4"+big"0.1"*im)
ndiag_indices = [(r, s) for r in 1//2:1//2:20 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
fields = vcat([Field(c, r = r, s = s) for (r, s) in ndiag_indices], diag_field)

function precompute_blocks(indices, fields; parity, precision=10)
    setprecision(BigFloat, floor(Int, 1.2*precision), base=10)
    Δmax = floor(Int, 0.75*precision)
    co = Correlation(Field(c, r=indices[1], s=indices[2]), Δmax)
    parity != 0 && (fields = filter(V -> V.diagonal || V.s >= 0, fields))
    f = V -> IBlock(co, V, parity=parity)
    spec = ChanSpec(co, fields, f)
    return @channels spec
end

function compute_diagblocks(specs, Ps, chan)
    c = specs.s.corr.fields[1].c
    return [IBlock(specs.s.corr, chan, Field(c, P=P)) for P in Ps]
end

# Solve crossing symmetry for given signature
# Optionnally for many different diagonal blocks in the channel
function solve(
        specs, signature;
        rmax=3, show=true, 
        Ps=nothing, diagchan=nothing,
        fix=nothing
    )
    specs = @channels filter(
        V -> V.diagonal && signature[chan] == 0 || !V.diagonal && V.r >= signature[chan],
        specs[chan]
    )
    if Ps === nothing
        res = solve_bootstrap(specs, fix=fix)
        if show
            Base.show(stdout, res.str_cst, rmax=rmax)
        end
    else
        diagblocks = compute_diagblocks(specs, Ps, diagchan)
        res = solve_bootstrap_manyP(specs, diagblocks)
    end
    return res
end

solve (generic function with 1 method)

## r = 0

In [16]:
ind = (0, sqrt(big"2")*(big"0.4"+big"0.2"*im))
blocks = precompute_blocks(ind, fields, parity=1, precision=40)
sol = solve(blocks, Sig(0, 0, 0), rmax=2);

 Channel s
┌─────────────────┬────────────────────────┬────────────────┐
│ Field           │ Structure constant     │ Relative error │
├─────────────────┼────────────────────────┼────────────────┤
│ (P=0.11+0.06im) │              1.0+0.0im │            0.0 │
│       (1//2, 0) │     0.416666+0.12037im │     2.7384e-41 │
│          (1, 0) │   0.251697-0.0896142im │    1.10052e-38 │
│          (1, 1) │ -0.00288987-0.315723im │    5.07092e-39 │
│       (3//2, 0) │  0.0631933-0.0402079im │    1.09577e-33 │
│    (3//2, 2//3) │  -0.0976787-0.273136im │    2.69897e-34 │
│          (2, 0) │  -0.032532+0.0304339im │    8.55505e-27 │
│       (2, 1//2) │ -0.231744+0.00741635im │     1.8528e-27 │
│          (2, 1) │   -0.134148-0.199107im │    1.81795e-27 │
└─────────────────┴────────────────────────┴────────────────┘
 Channel t
┌─────────────────┬────────────────────────┬────────────────┐
│ Field           │ Structure constant     │ Relative error │
├─────────────────┼────────────────────────┼────

In [28]:
display(c)
display(sol.correlation)
# show(stdout, sol.str_cst, rmax=2, backend = :latex)

c = -4.370934256055363321799307958477508650519031141868512110726643598615916955017481 + 7.344498269896193771626297577854671280276816608996539792387543252595155709342747im, β = -1.663780661615405939766692616717291857140790441619938909619623221165567621720136 + 0.4159451654038514849416731541793229642851976104049847274049058052913919054300341im

NonChiralCorrelation{Complex{BigFloat}} with external fields
< (1, 2) >

In [8]:
β = c.β
ind = (0, β * (β - 2/β)) # P_(1, 2)
blocks = precompute_blocks(ind, fields, parity=1, precision=30)
solve(blocks, Sig(0, 0, 0), rmax=2);

 Channel s
┌─────────────────┬─────────────────────────┬────────────────┐
│ Field           │ Structure constant      │ Relative error │
├─────────────────┼─────────────────────────┼────────────────┤
│ (P=0.11+0.06im) │               1.0+0.0im │            0.0 │
│       (1//2, 0) │      0.355233-0.12359im │    1.10493e-30 │
│          (1, 0) │     0.202314-0.121132im │    8.50096e-28 │
│          (1, 1) │  -0.0591978-0.0718335im │    4.67448e-28 │
│       (3//2, 0) │   0.0700839-0.0659127im │    8.64701e-23 │
│    (3//2, 2//3) │   -0.144613-0.0452004im │    3.01278e-23 │
│          (2, 0) │ 0.0413411+0.000518487im │    4.54786e-16 │
│       (2, 1//2) │   -0.110954+0.0600867im │    1.26047e-16 │
│          (2, 1) │  -0.143968+0.00561456im │     1.0614e-16 │
└─────────────────┴─────────────────────────┴────────────────┘
 Channel t
┌─────────────────┬─────────────────────────┬────────────────┐
│ Field           │ Structure constant      │ Relative error │
├─────────────────┼──────────────

## $r = 1, s = 0$

In [32]:
blocks = precompute_blocks((1, 0), fields, parity=1, precision=80);

### Signature $(0, 0, 0)$

In [53]:
using BarnesDoubleGamma
setprecision(BigFloat, 20, base=10)
β = c.β
DG = DoubleGamma(β)
V = Field(c, r=3//2, s=2//3)
V1 = Field(c, r=1, s=0)
V2 = Field(c, r=0, s=β^2)
println(V1[:left].Δ ≈ V2[:left].Δ)
println(LoopModels.Cref(V2, V, V) / LoopModels.Cref(V1, V, V))
println(gamma(1))

true
2.000000000630581425861 + 6.185447576316674431699e-10im
1.0


In [72]:
ϵ = 1e-8
V0 = Field(c, r=0, s=1//2)
V = Field(c, r=1, s=0)
V1 = Field(c, r=1, s=0)
V2 = Field(c, r=0, s=β^2)
println(LoopModels.Cref(V2, V0, V) / LoopModels.Cref(V1, V0, V))
println(LoopModels.Bref(V1, DG))
println(2 / DG(2β)^2 / DG(β + 1 / β)^2)

1.000000000000000000027 + 0.0im
101.1813198816819643318 + 28.95437238763036572509im
101.1801640727629699812 + 28.95462018026876427779im


In [59]:
gamma(1/2) / sqrt(π)

1.0000000000000002

In [33]:
sol = solve(blocks, Sig(0, 0, 0), rmax=8);

 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.11+0.06im) │                  1.0+0.0im │            0.0 │
│       (1//2, 0) │           6.2154-4.35819im │            0.0 │
│          (1, 0) │          5.84666-10.1272im │            0.0 │
│          (1, 1) │         -4.14065-1.37768im │            0.0 │
│       (3//2, 0) │          1.45015-8.15207im │            0.0 │
│    (3//2, 2//3) │          -12.1188+4.3916im │            0.0 │
│          (2, 0) │          3.97543-3.49046im │            0.0 │
│       (2, 1//2) │         -5.33376+15.1759im │            0.0 │
│          (2, 1) │         -12.8379+12.8515im │            0.0 │
│       (5//2, 0) │          26.1133-7.43759im │            0.0 │
│    (5//2, 2//5) │          5.91677+12.6273im │            0.0 │
│    (5//2, 4//5) │          5.93916+12.5801im │            0.0 │

In [34]:
sol

BootstrapSystem{Complex{BigFloat}}
< (1, 0) >
Number of positions: 87
Matrix size: (174, 167)


In [23]:
# show(stdout, sol.str_cst, backend=:latex, rmax=3)

## $r=1, s = 2$

In [ ]:
blocks = precompute_blocks((1, 2), fields, parity=0, precision=60);
sol = solve(blocks, Sig(0, 0, 0));

 Channel s
┌─────────────────┬──────────────────────┬────────────────┐
│ Field           │ Structure constant   │ Relative error │
├─────────────────┼──────────────────────┼────────────────┤
│ (P=0.11+0.06im) │            1.0+0.0im │            0.0 │
│       (1//2, 0) │   -4.40242+4.03791im │    1.01343e-29 │
│          (1, 0) │   -10.1201+11.9725im │    3.31304e-27 │
│          (1, 1) │    7.34646+7.71471im │    7.29649e-27 │
│       (3//2, 0) │   -6.96244+12.7403im │    7.16963e-23 │
│    (3//2, 2//3) │    29.3043+3.40064im │    9.87459e-23 │
│   (3//2, -2//3) │    17.0082-3.12202im │    6.41755e-23 │
│          (2, 0) │   -10.6338+3.27378im │    4.17103e-16 │
│      (2, -1//2) │    18.1994-22.7161im │    1.74474e-16 │
│       (2, 1//2) │    29.2332-26.2693im │    4.09395e-16 │
│          (2, 1) │     50.421-10.7106im │     3.4283e-16 │
│       (5//2, 0) │   -64.0131-14.2118im │     9.39071e-8 │
│   (5//2, -2//5) │   0.148746-30.5888im │      1.2882e-7 │
│    (5//2, 2//5) │    4.0134

In [26]:
# compare with V_{P_{(1, 2)}}
β = c.β
blocks = precompute_blocks((0, β * (β - 2/β)), fields, parity=0, precision=30)
solve(blocks, Sig(0, 0, 0))

 Channel s
┌─────────────────┬─────────────────────────┬────────────────┐
│ Field           │ Structure constant      │ Relative error │
├─────────────────┼─────────────────────────┼────────────────┤
│ (P=0.11+0.06im) │               1.0+0.0im │            0.0 │
│       (1//2, 0) │      0.355233-0.12359im │    2.49264e-30 │
│          (1, 0) │     0.202314-0.121132im │    1.32465e-27 │
│          (1, 1) │  -0.0591978-0.0718335im │    2.37294e-27 │
│       (3//2, 0) │   0.0700839-0.0659127im │    1.07924e-22 │
│    (3//2, 2//3) │   -0.144613-0.0452004im │    4.93023e-23 │
│   (3//2, -2//3) │   -0.144613-0.0452004im │    2.87732e-23 │
│          (2, 0) │ 0.0413411+0.000518487im │    7.49363e-16 │
│      (2, -1//2) │   -0.110954+0.0600867im │    1.66018e-16 │
│       (2, 1//2) │   -0.110954+0.0600867im │    1.58187e-16 │
│          (2, 1) │  -0.143968+0.00561456im │    1.22106e-16 │
│       (5//2, 0) │     0.13455+0.0783828im │     1.49143e-7 │
│   (5//2, -2//5) │  -0.0288292+0.0747277im 

BootstrapSystem{Complex{BigFloat}}
< (P=0.27+-0.35im) >
Number of positions: 58
Matrix size: (116, 110)


## r = 2

In [20]:
blocks = precompute_blocks((2, 0), fields, parity=1, precision=25);

### Signature $(\frac12, \frac12, \frac12)$

In [21]:
solve(blocks, Sig(1//2, 1//2, 1//2), rmax=3//2);

 Channel s
┌──────────────┬────────────────────┬────────────────┐
│ Field        │ Structure constant │ Relative error │
├──────────────┼────────────────────┼────────────────┤
│    (1//2, 0) │          1.0+0.0im │            0.0 │
│       (1, 0) │ -34.9208-45.5559im │     3.6251e-13 │
│       (1, 1) │  31.0888+25.0735im │    3.59439e-13 │
│    (3//2, 0) │ -589.454+35.4061im │    6.61775e-10 │
│ (3//2, 2//3) │  277.297-26.5527im │     1.33174e-9 │
└──────────────┴────────────────────┴────────────────┘
 Channel t
┌──────────────┬────────────────────┬────────────────┐
│ Field        │ Structure constant │ Relative error │
├──────────────┼────────────────────┼────────────────┤
│    (1//2, 0) │  1.0+4.58897e-14im │    1.54374e-14 │
│       (1, 0) │ -34.9208-45.5559im │    6.25701e-14 │
│       (1, 1) │  31.0888+25.0735im │    1.25781e-13 │
│    (3//2, 0) │ -589.454+35.4061im │    1.15658e-11 │
│ (3//2, 2//3) │  277.297-26.5527im │    5.00024e-11 │
└──────────────┴────────────────────┴──────

### Signature $(0, 0, 0)$, $D^{(s)}_{(\frac12, 0)} = 0$

In [22]:
fix = [(:s, diag_field, 1.0), (:s, Field(c, r=1//2, s=0), 0.0)]
solve(blocks, Sig(0, 0, 0), fix=fix, rmax=3);

 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.11+0.06im) │                  1.0+0.0im │            0.0 │
│       (1//2, 0) │                  0.0+0.0im │            0.0 │
│          (1, 0) │         -934.438+210.945im │    1.98011e-13 │
│          (1, 1) │         -597.189+296.177im │    1.83608e-13 │
│       (3//2, 0) │          1538.45+4136.88im │     1.71271e-9 │
│    (3//2, 2//3) │          1587.23+3847.92im │     1.26839e-9 │
│          (2, 0) │          12114.5-2452.02im │     0.00136152 │
│       (2, 1//2) │          3438.54-30959.3im │    0.000479461 │
│          (2, 1) │          11198.4-2765.52im │     0.00123378 │
│       (5//2, 0) │  4.07551e+08+8.40487e+08im │        1.10377 │
│    (5//2, 2//5) │  6.43446e+08+7.50261e+08im │        1.10006 │
│    (5//2, 4//5) │  7.85953e+08+6.64319e+08im │        1.12477 │

## r = 3

In [23]:
blocks = precompute_blocks((3, 0), fields, parity=1, precision=20);

### Signature $(1, 1, 1)$

In [24]:
solve(blocks, Sig(1, 1, 1), rmax=2);

 Channel s
┌──────────────┬──────────────────────┬────────────────┐
│ Field        │ Structure constant   │ Relative error │
├──────────────┼──────────────────────┼────────────────┤
│       (1, 0) │            1.0+0.0im │            0.0 │
│       (1, 1) │ -1.12641-0.0491036im │    8.92952e-14 │
│    (3//2, 0) │                    0 │       0.601382 │
│ (3//2, 2//3) │                    0 │        1.34188 │
│       (2, 0) │    10728.0+15111.3im │        1.17e-6 │
│    (2, 1//2) │   -10603.0-14053.7im │     1.04044e-6 │
│       (2, 1) │    10112.1+11149.5im │     9.93827e-7 │
└──────────────┴──────────────────────┴────────────────┘
 Channel t
┌──────────────┬──────────────────────┬────────────────┐
│ Field        │ Structure constant   │ Relative error │
├──────────────┼──────────────────────┼────────────────┤
│       (1, 0) │    1.0-5.91106e-14im │    8.71426e-14 │
│       (1, 1) │ -1.12641-0.0491036im │    1.73418e-13 │
│    (3//2, 0) │                    0 │        1.90731 │
│ (3//2, 

### Signature $(0, 0, 0)$, $D^{(s)}_{(1, 0)} = 0$, $D^{(s)}_{(2, 0)} = 0$

In [25]:
fix = [(:s, diag_field, 1), (:s, Field(c, r=1, s=0), 0), (:s, Field(c, r=2, s=0), 0)]
solve(blocks, Sig(0, 0, 0), fix=fix, rmax=2);

 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.11+0.06im) │                  1.0+0.0im │            0.0 │
│       (1//2, 0) │          13.3099+20.2186im │     1.16449e-6 │
│          (1, 0) │                  0.0+0.0im │            0.0 │
│          (1, 1) │         -8809.16-14776.1im │     1.17125e-6 │
│       (3//2, 0) │     6.82216e+06+460636.0im │     1.69767e-6 │
│    (3//2, 2//3) │ -4.92108e+06-6.33165e+06im │     5.84724e-7 │
│          (2, 0) │                  0.0+0.0im │            0.0 │
│       (2, 1//2) │  6.38691e+06+2.81696e+06im │     8.62956e-7 │
│          (2, 1) │ -1.03563e+08+4.48727e+08im │     1.08336e-6 │
└─────────────────┴────────────────────────────┴────────────────┘
 Channel t
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relat